### Web Scraper

This is a web scraper that can scrape automatically using selenium and chrome webdriver. We will be using keepa.com, an amazon price tracking website as our data source. Because the price data is contained in an interactive graph, we cannot scrape data simply by scraping html data using beautifulsoup.The logic behind this web scraper is we simulate human looking through the data by mouse movement and we scrape date and price data when the mouse is at that particular position. By switching urls and XPath of location of data we need, we can scrape any keepa.com website. 

The example below is an example of scraping data for a RTX 2080 GPU. Changing urls and other variables if other GPUs need to be scraped.

In [24]:
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium import webdriver
import datetime as dt
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains 


In [25]:
# Opening the connection and grabbing the page
my_url = 'https://keepa.com/#!product/1-B07W3P4PC2'
option = Options()
option.headless = False
option.add_argument("--incognito")
driver = webdriver.Chrome(options=option)

driver.get(my_url)
driver.set_window_position(0, 0)
driver.set_window_size(1024, 768)

In [26]:
# 等待到加载出走势图
element = WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH, "//canvas[@class='flot-overlay']")))

action = ActionChains(driver) 

# “全部”按钮
button_all = driver.find_element(By.XPATH, "//*[@class='legendTable']/tbody/tr/td/table/tbody/tr[last()]/td[2]")
action.move_to_element(button_all).click(button_all).perform()

# 指针移到图标最左端
action.move_to_element_with_offset(element, 50, 200).perform()
action.move_to_element_with_offset(element, 44, 200).perform()

In [27]:
# 测试读取价格
#XPath is being used to identify the location
value_new = driver.find_element(By.XPATH, "//*[@id='flotTip1']").text
value_2nd = driver.find_element(By.XPATH, "//*[@id='flotTip2']").text
date = driver.find_element(By.XPATH, "//*[@id='flotTipDate']").text

print(value_new)
print(value_2nd)
print(date)

全新品
$ 889.99

星期四, 八月 8 16:42


In [382]:
# 日期处理函数
def date_process(date_hist):
    chin2eng = {'一月':'1', '二月':'2', '三月':'3', '四月':'4', '五月':'5', '六月':'6', \
                '七月':'7', '八月':'8', '九月':'9', '十月':'10', '十一月':'11', '十二月':'12'}
    date_num = []
    date_processed = []
    year = dt.datetime.now().year
    
    # 文字转数字
    for date in date_hist:
        date_num.append(chin2eng[date.split(' ')[1]] + '-' + date.split(' ')[2]) 

    # 增加年份
    date_num.reverse()
    for i in range(0, len(date_num)):
        date_processed.append(str(year) + '-' + date_num[i])
        if date_num[i][0:2] == '1-' and date_num[i+1][0:3] == '12-':
            year -= 1
    date_processed.reverse()

    return date_processed

In [394]:
# 售价处理函数
def price_process(price_hist):
    price_processed = []

    for price in price_hist:
        if '' == price:
            price_processed.append(float('nan'))
        else:
            price_processed.append(price.split(' ')[1].replace(',', ''))

    return price_processed

In [385]:
price_new_hist = []
price_2nd_hist = []
date_hist = []

for step, x_now in enumerate(range(44, 590)):

    # 向右移动指针一步
    action.move_to_element_with_offset(element, x_now, 200).perform()
    
    # 获得售价及日期
    value_new = driver.find_element(By.XPATH, "//*[@id='flotTip1']").text
    value_2nd = driver.find_element(By.XPATH, "//*[@id='flotTip2']").text
    date = driver.find_element(By.XPATH, "//*[@id='flotTipDate']").text

    # 日期为空的数据跳过
    if '' == date:
        continue

    price_new_hist.append(value_new)
    price_2nd_hist.append(value_2nd)
    date_hist.append(date)

    # 输出日志
    if step % 10 == 0:
        print('No.{}  {}%'.format(step, (x_now-44)/(590-44)))


No.0  日期：星期四, 八月 8 17:13  全新价：全新品
$ 889.99  二手价：
No.1  日期：星期六, 八月 10 10:51  全新价：全新品
$ 889.42  二手价：
No.2  日期：星期一, 八月 12 4:28  全新价：全新品
$ 889.00  二手价：
No.3  日期：星期二, 八月 13 22:05  全新价：全新品
$ 789.99  二手价：
No.4  日期：星期四, 八月 15 15:42  全新价：全新品
$ 789.99  二手价：
No.5  日期：星期六, 八月 17 9:20  全新价：全新品
$ 789.00  二手价：
No.6  日期：星期一, 八月 19 2:57  全新价：全新品
$ 789.00  二手价：
No.7  日期：星期二, 八月 20 20:34  全新价：全新品
$ 829.99  二手价：
No.8  日期：星期四, 八月 22 14:11  全新价：全新品
$ 799.99  二手价：
No.9  日期：星期六, 八月 24 7:49  全新价：全新品
$ 799.99  二手价：
No.10  日期：星期一, 八月 26 1:26  全新价：全新品
$ 829.99  二手价：
No.11  日期：星期二, 八月 27 19:03  全新价：全新品
$ 799.99  二手价：
No.12  日期：星期四, 八月 29 12:40  全新价：全新品
$ 829.99  二手价：
No.13  日期：星期六, 八月 31 6:18  全新价：全新品
$ 829.99  二手价：
No.14  日期：星期日, 九月 1 23:55  全新价：全新品
$ 829.99  二手价：
No.15  日期：星期二, 九月 3 17:32  全新价：全新品
$ 829.99  二手价：
No.16  日期：星期四, 九月 5 11:09  全新价：全新品
$ 869.99  二手价：
No.17  日期：星期六, 九月 7 4:47  全新价：全新品
$ 839.42  二手价：非全新品
$ 799.99
No.18  日期：星期日, 九月 8 22:24  全新价：全新品
$ 869.99  二手价：非全新品
$ 799.99
No.19  日期：星期二, 九月 10 16:01  

In [399]:
date_processed = date_process(date_hist)
price_new_processed = price_process(price_new_hist)
price_2nd_processed = price_process(price_2nd_hist)

df = pd.DataFrame({'date':date_processed, 'price_new':price_new_processed, 'price_2nd':price_2nd_processed,})

df.head(3)

,date,price_new,price_2nd
0,2019-8-8,889.99,NaN
1,2019-8-10,889.42,NaN
2,2019-8-12,889.00,NaN
